# EPUG on Numpy Basics

*by Jonas Hartmann on 31.03.2020*

### Numpy Do Numb3rs Good!

In [ ]:
import numpy as np

In [ ]:
# Let's say we have some samples and some measurements for each sample

data = [[4,5,1,6,2,4,6,7,8,3,1],   # Sample 1
        [8,8,1,5,7,3,9,3,5,7,1],   # Sample 2 
        [7,6,8,3,3,6,4,5,5,7,3]]   # Sample 3

data

In [ ]:
# Let's normalize the data by dividing each measurement by the sample's maximum

data_normed = []
for sample in data:
    max_measure = max(sample)
    norm_measure = []
    for m in sample:
        norm_measure.append(m / max_measure)
    data_normed.append(norm_measure)
    
print(data_normed)

In [ ]:
# Now let's do the same in numpy

data = np.array(data)

data_normed = data / np.max(data, axis=1, keepdims=True)

print(data_normed)

# ->> Much simpler code and a substantial speed improvement!

#### *Notes from live session:*

In [ ]:
# By default, numpy attempts to broadcast along the last axis

# Given these shapes...
print('1:', data.shape)
print('2:', np.max(data, axis=1).shape)

# ...broadcasting will fail:
#data_normed = data / np.max(data, axis=1)

# If the last axis was consistent...
print('3:', data.T.shape)
print('4:', np.max(data, axis=1).shape)

# ...it would work (but the result is transposed)
data_normed = data.T / np.max(data, axis=1)

# There are several options for ensuring that broadcasting happens along the correct axis
data_normed = data / np.max(data, axis=1, keepdims=True)  # Keeps the original dimensionality
data_normed = data / np.max(data, axis=1).reshape((-1,1)) # Creates a view of appropriate dimensionality
data_normed = data / np.max(data, axis=1)[:, np.newaxis]  # Adds in another dimension
data_normed = data / np.max(data, axis=1)[:, None]        # Same as np.newaxis (shorter but less expressive)

# All of these produce this shape:
print('5:', np.max(data, axis=1)[:, None].shape)

### Working With Numpy Arrays

In [ ]:
# Numpy array properties

print(data.size)
print(data.ndim)
print(data.shape)

print(data.dtype)
print(data_normed.dtype)

In [ ]:
# Numpy array operations applied to entire arrays

print(data.sum(), np.sum(data) )  # These are equivalent
print(data.mean(), np.mean(data) )
print(np.percentile(data, 95) )  # Not all np functions have a method equivalent

In [ ]:
# Numpy array operations applied to specific axes (=dimensions)

print(data.sum(axis=1), np.sum(data, axis=1))
print(data.sum(axis=0), np.sum(data, axis=0))

# Note that the dimension specified is the one over which the operation
# "accumulates", so it's the dimension that has "disappeared" in the results.

# Also, note that you can accumulate over multiple dimensions using
# tuples as the axis argument.
d = np.random.randint(0, 10, size=(5,10,15))  # A random 3-dimensional array
d.sum(axis=(1,2))  # Sums up both the 2nd and 3rd axis/dimension

In [ ]:
# Calculations with numpy arrays apply element-wise

print(data)

print(data + 100)
print(data * 3)

print(data + data)

In [ ]:
# Numpy arrays can be "sliced" or "indexed into"

print(data[0])
print(data[:,0])
print(data[:,2:6])

In [ ]:
# Comparisons (boolean operations) apply element-wise

print(data > 5)

print(np.all(data > 5))
print(np.all(data > 0))

In [ ]:
# Boolean numpy arrays can function as masks for indexing

mask = data > 5
print(mask.dtype)

data[mask]

In [ ]:
# Doing the same with classical looping...

above_5 = []
for sample in data:
    for m in sample:
        if m > 5:
            above_5.append(m)

above_5

In [ ]:
# Numpy plays nice with many scientific computing tools!

# Numerical tools
from scipy.spatial import distance as dist
dist.squareform(dist.pdist(data))

In [ ]:
# Statistical tools
from scipy import stats
stats.pearsonr(data[0], data[1])

In [ ]:
# Visualization tools
import matplotlib.pyplot as plt

In [ ]:
for sample in data:
    plt.plot(sample)
plt.show()

In [ ]:
#### Note from live session: some additional indexing tricks

# Ellipsis indexing
d = np.random.randint(0, 10, size=(5,10,15,20))  # A random 4-dimensional array
d[:,:,:,0]   # This is the same...
d[...,0]     # ...as this!
d[0,:,:,0]   # This is the same...
d[0,...,0]   # ...as this

# Ellipsis indexing is useful when the number of dimensions may vary
d[...,0]     # Indexes first element of last dimension
data[...,0]  # Also indexes first element of last dimension (despite different ndim!)

# Slice objects provide additional flexibility as they can be assigned to variables and
# hence can be constructed and operated on outside of the actual indexing operation
d[1:4,0,0,0]         # This is the same...
d[slice(1,3),0,0,0]  # ...as this
d[:,0,0,0]            # This is the same
d[slice(None),0,0,0]  # ...as this
#s = :           # This wouldn't work!
s = slice(None)  # But this does!

# Numpy also supplies a custom slice object that can handle multiple dimensions
d[slice(1,5),0,0,0]  # This is the same...
d[np.s_[1:4],0,0,0]     # ...as this
d[:,...,0]         # This is the same...
d[np.s_[:,...,0]]  # ...as this
s = np.s_[:,...,0]
print(s)

### Arrays Everywhere

#### Microscopy images are arrays

In [ ]:
from skimage import io
img = io.imread(r'data/nuclei_DAPI_confocal.tif')

print(img.shape)
print(img.dtype)

In [ ]:
plt.imshow(img, cmap='gray')
plt.show()

In [ ]:
# Very basic thresholding
mask = img > np.mean(img)

In [ ]:
plt.imshow(mask, cmap='gray')
plt.show()

In [ ]:
# Rough estimate of mean nuclear intensity
np.mean(img[mask])

See [this self-explanatory online course](https://github.com/WhoIsJack/python-bioimage-analysis-tutorial) if you want to get into image analysis with python!

In [ ]:
#### Note from live session: always be aware of your data types!

# Most microscopy images are saved in `uint8` format.
# This is very memory efficient (only 8bit per number)
# but it can lead to weird results if used incorrectly!

# u -> unsigned, i.e. no negative numbers
# int -> integers, i.e. whole numbers (0, 1, 2)
# 8 -> 8bit, i.e. numbers from 0 to 255

# One common source of problems is that numbers above 255 or
# below 0 "wrap around". Check this out:
img_weird = img + 100
plt.imshow(img_weird, cmap='gray')
plt.show()

#### Time course data are arrays

In [ ]:
tc = np.load(r'data/totally_real_timecourse_data.npy')

print(tc.shape)
print(tc.dtype)

In [ ]:
for track in tc:
    plt.plot(track)
plt.show()

In [ ]:
# Norm maxima again, as in the initial examples

tc_normed = tc / np.max(tc, axis=1, keepdims=True)

for track in tc_normed:
    plt.plot(track)
plt.show()

In [ ]:
# BONUS: Shift tracks in time to overlay maximum slope

tc_time = np.ones_like(tc) * np.arange(tc.shape[1])

for t in range(tc.shape[0]):
    smoothed = np.convolve(tc_normed[t], np.ones(5), mode='valid')  # Smoothen to make max detection robust
    max_slope_idx = np.argmax(np.diff(smoothed)) + 2                # Detect max slope (+2 bc smoothed is shorter!)
    tc_time[t] -= max_slope_idx                                     # Shift the time accordingly
    
for time, track in zip(tc_time, tc_normed):
    plt.plot(time, track)
plt.show()

# ->> Turns out these tracks varied only in their timing and maximum value. 
#     Other than that, their dynamics are essentially identical!

#### Feature spaces are arrays

In [ ]:
# Grab a classical machine learning example dataset from scikit-learn
from sklearn.datasets import load_iris
iris = load_iris()

# What do we have here 
print(iris.data.shape)     # The actual data; 150 samples by 4 features
print(iris.feature_names)  # The 4 features that were measured (they are plant flower measures)
print(iris.target.shape)   # The classification of the 150 samples into 3 plant species
print(iris.target_names)   # The names of the 3 plan species

In [ ]:
# Run a Principal Component Analysis
from sklearn.decomposition import PCA
iris_pca = PCA().fit_transform(iris.data)

plt.scatter(iris_pca[:,0], iris_pca[:,1], c=iris.target)
plt.xlabel('PC 1')
plt.ylabel('PC 2')
plt.show()

# ->> Looks like PC 1 distinguishes the species quite well, especially setosa
#     on the left, whereas PC 2 captures something unrelated to species

#### Your data are arrays!

...probably. ;p

There are cases where arrays are imperfect, such as when there is a different number of measurements/timepoints/whatever for each sample. In such cases, don't hesitate to combine numpy with classical python, e.g. using a list of arrays or a dict of arrays.

Another common case where pure numpy arrays are suboptimal is if you have different data types (e.g. a machine learning dataset with some numerical features, some boolean features, some string features). In such cases, have a look at the pandas module and its dataframe objects.

### To learn more...

...I recommend starting some toy projects! For example, try to analyze some data you're interested in (from your science, from your holiday pictures, from something in the news, etc...) or try to come up with a simulation for something you find interesting (e.g. a simple simulation of evolution, or something based on differential equations if you're familiar with those, etc...). This is a great way of learning in my experience. 

Alternatively, check out the links to course materials and tutorials on the Bio-IT webpage [here](https://bio-it.embl.de/online-learning/) and [here](https://bio-it.embl.de/coding-club/curated-tutorials/) or try your hand at some coding challenges such as [Advent of Code](https://adventofcode.com/).